In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
registry_patient_selection_path = '/Users/jk1/temp/opsum_extration_output/high_frequency_data_patient_selection_with_details.csv'
output_path = '/Users/jk1/temp/opsum_extration_output'

In [ ]:
registry_data_df = pd.read_csv(registry_patient_selection_path, dtype=str)

In [ ]:
registry_data_df['registry_stroke_onset'] = pd.to_datetime(pd.to_datetime(registry_data_df['Onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + registry_data_df['Onset time'], format='%d-%m-%Y %H:%M').dt.strftime('%d.%m.%Y %H:%M')

registry_data_df['registry_admission_date'] = pd.to_datetime(pd.to_datetime(registry_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + registry_data_df['Arrival time'], format='%d-%m-%Y %H:%M').dt.strftime('%d.%m.%Y %H:%M')

In [ ]:
registry_data_df['case_admission_id'] = registry_data_df['patient_id'].astype(str) \
                                 + registry_data_df['EDS_last_4_digits'].astype(str) \
                                 + '_' + pd.to_datetime(registry_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d%m%Y').astype(str)

In [ ]:
len(registry_data_df['case_admission_id'].unique())

In [ ]:
import pandas as pd
eds_df = pd.read_csv('/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20211110/eds_j1.csv', delimiter=';', dtype=str)

In [ ]:
from preprocessing.utils import create_ehr_case_identification_column

eds_df['case_admission_id'] = create_ehr_case_identification_column(eds_df)

In [ ]:
len(eds_df['case_admission_id'].unique())

In [ ]:
eds_df[eds_df['case_admission_id'] == '16152643_2643']

Restrict to patient selection

In [ ]:
restricted_eds_df = eds_df[eds_df['case_admission_id'].isin(registry_data_df['case_admission_id'])]


In [ ]:
joined_df = pd.merge(registry_data_df, restricted_eds_df, on=['case_admission_id'], how='left')

## Find patients with differing admission dates

Difference between stroke registry admission (registry_admission_date) and DPI admission (date_from)

In [ ]:
joined_df['registry_arrival_to_date_from_days'] = (pd.to_datetime(joined_df['date_from'], format='%d.%m.%Y %H:%M') - pd.to_datetime(joined_df['Arrival at hospital'], format='%Y%m%d')).dt.days


Find patients with a difference of over 10 days in admission date (thus 14 days period does not cover 4 days of admission)

In [ ]:
differing_starting_date_df = joined_df[(joined_df['registry_arrival_to_date_from_days'] > 0) |(joined_df['registry_arrival_to_date_from_days'] < -10)]

In [ ]:
differing_starting_date_df[['case_admission_id', 'date_from', 'Arrival at hospital', 'registry_arrival_to_date_from_days', 'registry_stroke_onset', 'Referral']]

In [ ]:
differing_starting_date_df.columns

In [ ]:
differing_starting_date_df.drop(['Unnamed: 0', 'Onset date', 'Onset time', 'nr', 'patient_id_y',
       'EDS_last_4_digits_y', 'DOB_y', 'begin_date', 'end_date',
       'death_date', 'death_hosp', 'eds_final_id', 'eds_final_begin',
       'eds_final_end', 'eds_final_patient_id', 'eds_final_birth',
       'eds_final_death', 'eds_final_birth_str', 'date_from', 'date_to', 'eds_manual'], axis=1, inplace=True)

In [ ]:
# differing_starting_date_df.to_csv(os.path.join(output_path, 'patients_with_wrong_eds_start.csv'), index=False)

Find patients with a difference of less than 10 days in admission date (thus 14 days period covers 4 days of admission)

In [ ]:
minor_differing_starting_date_df = joined_df[(joined_df['registry_arrival_to_date_from_days'] <= -2) &(joined_df['registry_arrival_to_date_from_days'] >= -10)]

In [ ]:
minor_differing_starting_date_df.drop(['Unnamed: 0', 'Onset date', 'Onset time', 'nr', 'patient_id_y',
       'EDS_last_4_digits_y', 'DOB_y', 'begin_date', 'end_date',
       'death_date', 'death_hosp', 'eds_final_id', 'eds_final_begin',
       'eds_final_end', 'eds_final_patient_id', 'eds_final_birth',
       'eds_final_death', 'eds_final_birth_str', 'date_from', 'date_to', 'eds_manual'], axis=1, inplace=True)

In [ ]:
minor_differing_starting_date_df

In [ ]:
# minor_differing_starting_date_df.to_csv(os.path.join(output_path, 'patients_with_slightly_shifted_eds_start.csv'), index=False)
